In [1]:
chr(0)

'\x00'

In [2]:
print(chr(0))

 


In [3]:
print(chr(0).__repr__())

'\x00'


In [4]:
test_string = "this is a test" + chr(0) + "string"

In [5]:
print(test_string)

this is a test string


In [6]:
test_string = "hello! こんにちは!"
utf8_encoded = test_string.encode("utf-8")

In [7]:
print(utf8_encoded)
print(list(utf8_encoded))

b'hello! \xe3\x81\x93\xe3\x82\x93\xe3\x81\xab\xe3\x81\xa1\xe3\x81\xaf!'
[104, 101, 108, 108, 111, 33, 32, 227, 129, 147, 227, 130, 147, 227, 129, 171, 227, 129, 161, 227, 129, 175, 33]


In [8]:
len(test_string), len(utf8_encoded)

(13, 23)

In [9]:
utf8_encoded.decode("utf-8")

'hello! こんにちは!'

In [10]:
utf16_encoded = test_string.encode("utf-16")
utf32_encoded = test_string.encode("utf-32")

In [11]:
print (len(list(utf8_encoded)))
print (len(list(utf16_encoded)))
print (len(list(utf32_encoded)))

23
28
56


In [12]:
test_string = "hello"
print(len(test_string))
print(len(list(test_string.encode("utf-8"))))

5
5


In [13]:
def decode_utf8_bytes_to_str_wrong(bytestring: bytes):
	return "".join([bytes([b]).decode("utf-8") for b in bytestring])

In [14]:
decode_utf8_bytes_to_str_wrong("hello".encode("utf-8"))

'hello'

In [18]:
list("hello".encode("utf-8"))

[104, 101, 108, 108, 111]

In [ ]:
# decode_utf8_bytes_to_str_wrong("hello! こんにちは!".encode("utf-8")) # Error

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe3 in position 0: unexpected end of data

In [16]:
bytes(list("こ".encode("utf-8"))[:-1]).decode("utf-8") # Error - probably no byte sequence can be prefix of any other?

UnicodeDecodeError: 'utf-8' codec can't decode bytes in position 0-1: unexpected end of data

In [45]:
PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""

In [46]:
import regex as re

In [47]:
re.findall(PAT, "some text that i'll pre-tokenize")

['some', ' text', ' that', ' i', "'ll", ' pre', '-', 'tokenize']

In [48]:
re.findall(PAT, "this is a piece of random text")

['this', ' is', ' a', ' piece', ' of', ' random', ' text']

In [49]:
re.findall(PAT, "this is a piece of random text.")

['this', ' is', ' a', ' piece', ' of', ' random', ' text', '.']

In [60]:
corpus = []
with open('tiny_corpus.txt', 'r') as f:
	for line in f.readlines():
		corpus.extend([word.strip() for word in line.split(' ')])

In [82]:
from collections import Counter

def bpe_tokenizer(corpus, num_merges):
	new_tokens = []

	# Create a dict mapping pretokens to frequency
	freq = Counter()

	for pretoken in corpus:
		freq[tuple(pretoken)] += 1

	# Loop through for num_merges
	for _ in range(num_merges):

		# Count the number of co-orccurences
		bigrams = Counter()

		for pretoken in freq.keys():
			for char_1, char_2 in zip(pretoken[:-1], pretoken[1:]):
				bigrams[(char_1, char_2)] += freq[pretoken]

		# Get the max one resolving ties lexicographically
		max_val = max(bigrams.values())
		max_bigrams = [bigram for bigram in bigrams if bigrams[bigram] == max_val ]

		# Append to new_tokens
		merge_rule = max(max_bigrams)
		new_tokens.append(''.join(merge_rule))

		# Create new dict mapping pretokens to frequency with merge
		new_freq = {}
		for pretoken, val in freq.items():
			i = 0
			new_pretoken = []
			while i < len(pretoken) - 1:
				char_1 = pretoken[i]
				char_2 = pretoken[i + 1]

				if char_1 == merge_rule[0] and char_2 == merge_rule[1]:
					new_pretoken.append(char_1 + char_2)
					i += 2
				else:
					new_pretoken.append(char_1)
					i += 1

			if i == len(pretoken) - 1:
				new_pretoken.append(pretoken[i])
			new_freq[tuple(new_pretoken)] = val
		
		# Replace existing mapping
		freq = new_freq
	return new_tokens

In [ ]:
bpe_tokenizer(corpus, 6)

# Making an assumption here that each character is mapping to 1 byte

['st', 'est', 'ow', 'low', 'west', 'ne']

In [1]:
with open('data/TinyStoriesV2-GPT4-train.txt', 'r') as f:
	lines = f.readlines()

In [3]:
len(lines[0])

1